# kNNアルゴリズムとは
k近傍法(k-NearestNeighbor)アルゴリズムは、近接性を使用して個々のデータポイントのグループ化に関する分類または予測を行うノンパラメトリックな教師あり学習分類器である。
kNNアルゴリズムは回帰問題にも分類問題にも利用できるが、通常は分類アルゴリズムとして使用される。
基本的なアイデアは多数決である。カテゴリ数が2つの場合には「多数決」カテゴリ数がそれ以上の場合には「相対多数」と呼ばれる。あるデータポイントからk番目までに近いポイントのデータラベルの相対多数によって、そのデータポイントのラベルを決定する。
kNNアルゴリズムは「遅延学習」と呼ばれる分類に属し、トレーニングを実行するのではなく、トレーニング・データセットの保存のみを実行する。これは分類または予測が行われているときにすべての計算が行われることを意味する。また、すべてのトレーニングデータをメモリーに乗せる必要があるため、インスタンス・ベース、またはメモリー・ベースの学習方法とも呼ばれる。これは他の分類器と比較してより多くのメモリとデータストレージを消費するアプローチである。

# kNNの計算：距離メトリクス
kNNアルゴリズムの目的は特定のデータポイントの最近傍を識別し、そのポイントにクラスラベルを割り当てることである。これを行うために調整する必要のあるハイパーパラメータは


ノンパラメトリック：与えられた母集団が何らかの分布に従っている前提がない
パラメトリック：与えられた母集団が何らかの分布に従っている前提がある（だいたい正規分布）
回帰問題：入力データから数値の予測を行う問題。この場合与えられたデータポイントがどのグループに属するかを予測する
分類問題：与えられたデータのデータポイントがそれぞれどこに属するか
遅延学習：計算が必要になるまで計算を行なわない

# コード

In [8]:
import numpy as np
from csv import reader
from math import sqrt, exp
from statistics import median
from random import shuffle

def load_csv(filename):
    dataset = []
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            dataset.append([float(x) for x in row])
    return dataset

def euclidean_distance(row1, row2):
    return sqrt(sum((r1 - r2) ** 2 for r1, r2 in zip(row1[1:], row2[1:])))

def get_neighbors(train, test_row, k):
    distances = [(train_row, euclidean_distance(test_row, train_row)) for train_row in train]
    distances.sort(key=lambda x: x[1])
    return distances[:k]

def weighted_knn(train, test_row, k, sigma):
    neighbors = get_neighbors(train, test_row, k)
    class_votes = {}
    for neighbor, distance in neighbors:
        weight = exp(- (distance ** 2) / (2 * (sigma ** 2)))
        class_votes[neighbor[0]] = class_votes.get(neighbor[0], 0) + weight
    return max(class_votes, key=class_votes.get)

def evaluate_knn(dataset, k, split_ratio=0.7, runs=10, weighted=False):
    accuracy_list = []
    for _ in range(runs):
        shuffle(dataset)
        train_size = int(len(dataset) * split_ratio)
        train, test = dataset[:train_size], dataset[train_size:]
        
        if weighted:
            distance_matrix = [euclidean_distance(a, b) for a in train for b in train if a != b]
            sigma = median(distance_matrix)
        
        correct = 0
        for test_row in test:
            prediction = weighted_knn(train, test_row, k, sigma) if weighted else get_neighbors(train, test_row, 1)[0][0][0]
            if prediction == test_row[0]:
                correct += 1
        accuracy_list.append(correct / len(test))
    
    mean_acc = np.mean(accuracy_list)
    std_acc = np.std(accuracy_list)
    return mean_acc, std_acc

# Example usage
filename = './hayuci13a/yXT_wine.csv'
dataset = load_csv(filename)
k_values = [1, 2, 3, 4, 5]

for k in k_values:
    mean_acc, std_acc = evaluate_knn(dataset, k, weighted=True)
    print(f'k={k}: Mean Accuracy={mean_acc:.3f}, Std Dev={std_acc:.3f}')


k=1: Mean Accuracy=0.726, Std Dev=0.049
k=2: Mean Accuracy=0.743, Std Dev=0.070
k=3: Mean Accuracy=0.709, Std Dev=0.051
k=4: Mean Accuracy=0.731, Std Dev=0.042
k=5: Mean Accuracy=0.706, Std Dev=0.058


In [9]:
from random import seed, randrange
from csv import reader
from math import sqrt

def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# Convert string columns to float and adjust class label position
def preprocess_dataset(dataset):
    for row in dataset:
        row[:] = [float(x) for x in row]  # Convert all to float
        row.append(row.pop(0))  # Move class label to the end

# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = [[min(col), max(col)] for col in zip(*dataset)]
    return minmax

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)-1):  # Exclude class label
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size and dataset_copy:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = sum(1 for a, p in zip(actual, predicted) if a == p)
    return correct / len(actual) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = []
    for fold in folds:
        train_set = [row for f in folds if f is not fold for row in f]
        test_set = [list(row) for row in fold]
        for row in test_set:
            row[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        scores.append(accuracy_metric(actual, predicted))
    return scores

# Calculate Euclidean distance
def euclidean_distance(row1, row2):
    return sqrt(sum((row1[i] - row2[i])**2 for i in range(len(row1)-1)))

# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
    distances = [(train_row, euclidean_distance(test_row, train_row)) for train_row in train]
    distances.sort(key=lambda tup: tup[1])
    return [distances[i][0] for i in range(num_neighbors)]

# Make a prediction with neighbors
def predict_classification(train, test_row, num_neighbors):
    neighbors = get_neighbors(train, test_row, num_neighbors)
    output_values = [row[-1] for row in neighbors]
    return max(set(output_values), key=output_values.count)

# kNN Algorithm
def k_nearest_neighbors(train, test, num_neighbors):
    return [predict_classification(train, row, num_neighbors) for row in test]

# Load and prepare data
seed(1)
filename = './hayuci13a/yXT_wine.csv'  # 修正されたデータセット名
dataset = load_csv(filename)
preprocess_dataset(dataset)  # データを変換
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

# Evaluate algorithm
n_folds = 5
num_neighbors = 5
scores = evaluate_algorithm(dataset, k_nearest_neighbors, n_folds, num_neighbors)
print(f'Scores: {scores}')
print(f'Mean Accuracy: {sum(scores)/len(scores):.3f}%')


Scores: [97.14285714285714, 91.42857142857143, 94.28571428571428, 94.28571428571428, 100.0]
Mean Accuracy: 95.429%
